# 2.3 TRANSFORMACIONES DEL DATASET RETAIL

En este notebook aplico la etapa de **Transformaciones** al archivo:

**`retail_sales_limpio.csv`**

Estas transformaciones se basan en lo visto en clase en el notebook `Pandas02.ipynb`,  
pero ahora aplicado únicamente al dataset del proyecto.

El objetivo es generar variables nuevas y dejar los datos en un formato más útil para:

- La etapa 2.4 (Normalización)
- La etapa 2.5 (EDA)
- Los modelos de Machine Learning (Regresión y Clasificación)

Al final de este notebook se genera el archivo:

**`retail_sales_transformado.csv`**


In [5]:
# Importar librerías principales
import pandas as pd

# Cargar el dataset limpio (está en la misma carpeta que este notebook)
df = pd.read_csv("retail_sales_limpio.csv")

# Vista rápida de los primeros registros
df.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


## 1. Revisión de la estructura del DataFrame

Antes de transformar, reviso:

- Nombres de columnas
- Dimensiones
- Tipos de datos actuales

Esto asegura que las transformaciones se apliquen sobre los nombres correctos.


In [6]:
# Ver nombres de columnas
df.columns


Index(['id_transaccion', 'fecha', 'id_cliente', 'genero', 'edad',
       'categoria_producto', 'cantidad', 'precio_unitario', 'monto_total'],
      dtype='object')

In [7]:
# Información general del DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_transaccion      1000 non-null   int64 
 1   fecha               1000 non-null   object
 2   id_cliente          1000 non-null   object
 3   genero              1000 non-null   object
 4   edad                1000 non-null   int64 
 5   categoria_producto  1000 non-null   object
 6   cantidad            1000 non-null   int64 
 7   precio_unitario     1000 non-null   int64 
 8   monto_total         1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


## 2. Conversión de tipos de datos

En esta etapa:

- Convierto la columna `fecha` a tipo `datetime`
- Me aseguro de que las columnas numéricas sean realmente numéricas:

  - `edad`
  - `cantidad`
  - `precio_unitario`
  - `monto_total`

Esto es similar a lo que hicimos en clase para poder hacer operaciones temporales y estadísticas.


In [8]:
# Conversión de 'fecha' a tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Conversión de columnas numéricas a tipo numérico (por si hubiera texto raro)
columnas_numericas = ['edad', 'cantidad', 'precio_unitario', 'monto_total']

df[columnas_numericas] = df[columnas_numericas].apply(pd.to_numeric, errors='coerce')

# Verificación final de tipos
df.dtypes


id_transaccion                 int64
fecha                 datetime64[ns]
id_cliente                    object
genero                        object
edad                           int64
categoria_producto            object
cantidad                       int64
precio_unitario                int64
monto_total                    int64
dtype: object

## 3. Creación de variables temporales

A partir de la columna `fecha` genero nuevas columnas:

- `anio`  → año de la compra
- `mes`   → mes en número
- `dia`   → día del mes
- `nombre_mes` → nombre del mes (enero, febrero, etc.)

Esto ayuda a analizar el comportamiento de las ventas en el tiempo (por mes, por año, etc.).


In [9]:
# Crear columnas con partes de la fecha
df['anio'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['dia'] = df['fecha'].dt.day

# Nombre del mes (ej. January, February...)
df['nombre_mes'] = df['fecha'].dt.month_name()

# Vista rápida de las nuevas columnas
df[['fecha', 'anio', 'mes', 'dia', 'nombre_mes']].head()


,fecha,anio,mes,dia,nombre_mes
0,2023-11-24,2023,11,24,November
1,2023-02-27,2023,2,27,February
2,2023-01-13,2023,1,13,January
3,2023-05-21,2023,5,21,May
4,2023-05-06,2023,5,6,May


## 4. Segmentación de clientes por rango de edad

Clasifico a los clientes en rangos de edad para poder analizar:

- Qué grupo gasta más
- Qué tipo de productos compra cada grupo

Defino los rangos así:

- `< 25`  → "Joven"
- `25–44` → "Adulto"
- `>= 45` → "Mayor"


In [10]:
# Función auxiliar para asignar rango de edad
def clasificar_edad(valor):
    if valor < 25:
        return 'Joven'
    elif valor < 45:
        return 'Adulto'
    else:
        return 'Mayor'

# Crear nueva columna con rango de edad
df['rango_edad'] = df['edad'].apply(clasificar_edad)

df[['edad', 'rango_edad']].head()


,edad,rango_edad
0,34,Adulto
1,26,Adulto
2,50,Mayor
3,37,Adulto
4,30,Adulto


## 5. Limpieza y normalización de texto

Para evitar problemas con mayúsculas/minúsculas y espacios:

- `genero` → lo paso a mayúsculas (`MALE`, `FEMALE`)
- `categoria_producto` → lo paso a formato tipo título (`Electronics`, `Clothing`, etc.)
- `id_cliente` → quito espacios en blanco por si existiera alguno

Esto ayuda a que los filtros y agrupaciones funcionen correctamente.


In [11]:
# Normalizar el género (mayúsculas sin espacios)
df['genero'] = df['genero'].astype(str).str.strip().str.upper()

# Normalizar la categoría de producto (tipo título)
df['categoria_producto'] = df['categoria_producto'].astype(str).str.strip().str.title()

# Asegurar que el ID de cliente no tenga espacios
df['id_cliente'] = df['id_cliente'].astype(str).str.strip().str.replace(" ", "")

df[['genero', 'categoria_producto', 'id_cliente']].head()


,genero,categoria_producto,id_cliente
0,MALE,Beauty,CUST001
1,FEMALE,Clothing,CUST002
2,MALE,Electronics,CUST003
3,MALE,Clothing,CUST004
4,MALE,Beauty,CUST005


## 6. Verificación de consistencia entre Cantidad, Precio_Unitario y Monto_Total

Teóricamente se cumple:

\[
Monto\_Total \approx Cantidad \times Precio\_Unitario
\]

Para validar, creo una columna auxiliar llamada `monto_calculado` y comparo contra `monto_total`.  
También agrego una columna `diferencia_monto` que muestra la diferencia entre ambos.


In [12]:
# Calcular el monto teórico basado en cantidad * precio_unitario
df['monto_calculado'] = df['cantidad'] * df['precio_unitario']

# Diferencia entre el monto de la columna original y el calculado
df['diferencia_monto'] = df['monto_total'] - df['monto_calculado']

# Vista rápida de las columnas involucradas
df[['cantidad', 'precio_unitario', 'monto_total', 'monto_calculado', 'diferencia_monto']].head()


,cantidad,precio_unitario,monto_total,monto_calculado,diferencia_monto
0,3,50,150,150,0
1,2,500,1000,1000,0
2,1,30,30,30,0
3,1,500,500,500,0
4,2,50,100,100,0


## 7. Clasificación de transacciones por nivel de compra

Creo una clasificación simple para el análisis:

- `nivel_compra`:

  - `"Baja"`      → monto_total < 200  
  - `"Media"`     → 200 ≤ monto_total < 800  
  - `"Alta"`      → monto_total ≥ 800  

- `es_compra_alta`:

  - `True` si la compra es "Alta"  
  - `False` en cualquier otro caso  

Esto será útil más adelante para la parte de clasificación (Regresión Logística).


In [13]:
# Función para clasificar el nivel de compra
def clasificar_compra(monto):
    if monto < 200:
        return 'Baja'
    elif monto < 800:
        return 'Media'
    else:
        return 'Alta'

# Crear columna con nivel de compra
df['nivel_compra'] = df['monto_total'].apply(clasificar_compra)

# Crear columna booleana para compras altas
df['es_compra_alta'] = df['nivel_compra'] == 'Alta'

df[['monto_total', 'nivel_compra', 'es_compra_alta']].head()


,monto_total,nivel_compra,es_compra_alta
0,150,Baja,False
1,1000,Alta,True
2,30,Baja,False
3,500,Media,False
4,100,Baja,False


## 8. Vista general del DataFrame transformado

En este punto, el dataset ya incluye:

- Variables temporales (`anio`, `mes`, `dia`, `nombre_mes`)
- Segmentación de clientes (`rango_edad`)
- Normalización de texto en `genero` y `categoria_producto`
- Columnas auxiliares de validación (`monto_calculado`, `diferencia_monto`)
- Variables de negocio (`nivel_compra`, `es_compra_alta`)


In [14]:
# Vista general de algunas columnas clave
df[['id_transaccion',
    'fecha',
    'id_cliente',
    'genero',
    'edad',
    'rango_edad',
    'categoria_producto',
    'cantidad',
    'precio_unitario',
    'monto_total',
    'nivel_compra',
    'es_compra_alta']].head()


,id_transaccion,fecha,id_cliente,genero,edad,rango_edad,categoria_producto,cantidad,precio_unitario,monto_total,nivel_compra,es_compra_alta
0,1,2023-11-24,CUST001,MALE,34,Adulto,Beauty,3,50,150,Baja,False
1,2,2023-02-27,CUST002,FEMALE,26,Adulto,Clothing,2,500,1000,Alta,True
2,3,2023-01-13,CUST003,MALE,50,Mayor,Electronics,1,30,30,Baja,False
3,4,2023-05-21,CUST004,MALE,37,Adulto,Clothing,1,500,500,Media,False
4,5,2023-05-06,CUST005,MALE,30,Adulto,Beauty,2,50,100,Baja,False


In [15]:
# Estadísticas descriptivas de las columnas numéricas después de las transformaciones
df[['edad', 'cantidad', 'precio_unitario', 'monto_total']].describe()


,edad,cantidad,precio_unitario,monto_total
count,1000.00000,1000.000000,1000.000000,1000.000000
mean,41.39200,2.514000,179.890000,456.000000
std,13.68143,1.132734,189.681356,559.997632
min,18.00000,1.000000,25.000000,25.000000
25%,29.00000,1.000000,30.000000,60.000000
50%,42.00000,3.000000,50.000000,135.000000
75%,53.00000,4.000000,300.000000,900.000000
max,64.00000,4.000000,500.000000,2000.000000


## 9. Exportación del dataset transformado

Finalmente, guardo el DataFrame resultante como:

**`retail_sales_transformado.csv`**

Este será el archivo base para:

- 2.4 Normalización
- 2.5 EDA
- Modelos de Machine Learning


In [16]:
# Exportar el DataFrame transformado a un nuevo archivo CSV
df.to_csv("retail_sales_transformado.csv", index=False)

print("✅ Archivo 'retail_sales_transformado.csv' generado correctamente.")


✅ Archivo 'retail_sales_transformado.csv' generado correctamente.


# Conclusiones de la etapa 2.3 — Transformaciones

En esta etapa realicé las transformaciones necesarias sobre el archivo `retail_sales_limpio.csv` para enriquecer el dataset y prepararlo para el análisis y los modelos de Machine Learning.

Los principales logros fueron:

- Conversión correcta de tipos de datos, especialmente la columna `fecha`.
- Creación de variables temporales (`anio`, `mes`, `dia`, `nombre_mes`) que permiten estudiar el comportamiento de las ventas a lo largo del tiempo.
- Segmentación de clientes por `rango_edad`, útil para entender el perfil de los compradores.
- Normalización de texto en `genero`, `categoria_producto` e `id_cliente` para evitar problemas en filtros y agrupaciones.
- Verificación de consistencia entre `cantidad`, `precio_unitario` y `monto_total` mediante las columnas `monto_calculado` y `diferencia_monto`.
- Definición de variables de negocio (`nivel_compra`, `es_compra_alta`) que servirán como base para la Regresión Logística y otros análisis.
- Generación del archivo final **`retail_sales_transformado.csv`**, que será utilizado en las siguientes etapas del proyecto (Normalización, EDA y Modelado).

Con esto se cumple el apartado de **Transformaciones** dentro de la metodología de Ciencia de Datos solicitada por el profesor.
